Lịch sử giá

In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Có thể bỏ nếu muốn xem trình duyệt
service = Service(r"C:\Users\dongh\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 15)


def crawl_data_from_page(url):
    driver.get(url)
    time.sleep(3)

    all_data = []

    for page in range(1, 500):
        try:
            print(f"Đang xử lý trang {page}...")

            # Click vào trang hiện tại (trừ trang 1 đã load sẵn)
            if page > 1:
                if "4" in url:
                    pagination_button = wait.until(
                        EC.element_to_be_clickable((By.XPATH, f"//div[@onclick='tuDoanh.handleChangePage({page})']"))
                    )
                else:
                    pagination_button = wait.until(
                        EC.element_to_be_clickable((By.XPATH, f"//div[@onclick='ownerCDL.handleChangePage({page})']"))
                    )
                pagination_button.click()
                time.sleep(2)  # Đảm bảo nội dung được cập nhật

            # Chờ bảng dữ liệu hiển thị
            wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            table = soup.find("table", {"id": "owner-contents-table"})
            rows = table.find_all("tr")

            for row in rows:
                cols = row.find_all("td")
                if not cols:
                    continue
                values = [td.text.strip().replace(",", "").replace("--", "") for td in cols]
                all_data.append(values)

        except Exception as e:
            print(f"Lỗi ở trang {page}: {e}")
            break

    driver.quit()
    return all_data


In [2]:
url_VNM_lsg  = "https://cafef.vn/du-lieu/lich-su-giao-dich-vnm-1.chn"
url_VNM_td = "https://cafef.vn/du-lieu/lich-su-giao-dich-vnm-4.chn#data"
url_VNM_kn = "https://cafef.vn/du-lieu/lich-su-giao-dich-vnm-3.chn#data"
url_VNM_tkdl = "https://cafef.vn/du-lieu/lich-su-giao-dich-vnm-2.chn#data"

url_MB_lsg  = "https://cafef.vn/du-lieu/lich-su-giao-dich-mbb-1.chn#data"
url_MB_td = "https://cafef.vn/du-lieu/lich-su-giao-dich-mbb-4.chn#data"
url_MB_kn = "https://cafef.vn/du-lieu/lich-su-giao-dich-mbb-3.chn#data"
url_MB_tkdl = "https://cafef.vn/du-lieu/lich-su-giao-dich-mbb-2.chn#data"

url_HPG_lsg  = "https://cafef.vn/du-lieu/lich-su-giao-dich-hpg-1.chn#data"
url_HPG_td = "https://cafef.vn/du-lieu/lich-su-giao-dich-hpg-4.chn#data"    
url_HPG_kn = "https://cafef.vn/du-lieu/lich-su-giao-dich-hpg-3.chn#data"
url_HPG_tkdl = "https://cafef.vn/du-lieu/lich-su-giao-dich-hpg-2.chn#data"

url_HAG_lsg  = "https://cafef.vn/du-lieu/lich-su-giao-dich-hag-1.chn#data"
url_HAG_td = "https://cafef.vn/du-lieu/lich-su-giao-dich-hag-4.chn#data"    
url_HAG_kn = "https://cafef.vn/du-lieu/lich-su-giao-dich-hag-3.chn#data"
url_HAG_tkdl = "https://cafef.vn/du-lieu/lich-su-giao-dich-hag-2.chn#data"

url_FPT_lsg  = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-1.chn#data"
url_FPT_td = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-4.chn#data"    
url_FPT_kn = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-3.chn#data"
url_FPT_tkdl = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-2.chn#data"

In [3]:
lsg_cols = ["Ngày", "Giá đóng cửa", "Giá điều chỉnh", "Thay đổi", "KL gd khớp lệnh"
            , "GT gd khớp lệnh","KL gd thỏa thuận", "GT gd thỏa thuận"
            , "Giá mở cửa", "Giá cao nhất", "Giá thấp nhất" ]
td_cols = ["Mã", "Ngày", "KL mua", "GT mua","KL bán", "GT bán"
           , "KL giao dịch ròng", "GT giao dịch ròng"]
kn_cols = ["Ngày", "Thay đổi", "KL giao dịch ròng", "GT giao dịch ròng", "KL mua"
           , "GT mua", "KL bán", "GT bán", "Room còn lại", "Đang sở hữu"]
tkdl_cols = ["Ngày", "Thay đổi", "Số lệnh mua", "Kl mua", "KLTB 1 lệnh mua"
             , "Số lệnh bán", "Kl bán", "KLTB 1 lệnh bán", "Kl ròng"]


VNM

In [7]:
def process_data(data, columns):
    df = pd.DataFrame(data, columns=columns)
    df["Ngày"] = pd.to_datetime(df["Ngày"], format="%d/%m/%Y", errors="coerce")
    df = df.dropna(subset=["Ngày"])
    return df

In [ ]:
df = process_data(hag_tkdl, tkdl_cols)
df[df.isna().any(axis=1)]


,Ngày,Thay đổi,KL giao dịch ròng,GT giao dịch ròng,KL mua,GT mua,KL bán,GT bán,Room còn lại,Đang sở hữu


In [50]:
len(df)

4073

In [51]:
df.to_csv("hagl_kn.csv", index=False, encoding="utf-8-sig")

In [42]:
vnm_tkdl = crawl_data_from_page(url_VNM_tkdl)

Đang xử lý trang 1...
Đang xử lý trang 2...
Đang xử lý trang 3...
Đang xử lý trang 4...
Đang xử lý trang 5...
Đang xử lý trang 6...
Đang xử lý trang 7...
Đang xử lý trang 8...
Đang xử lý trang 9...
Đang xử lý trang 10...
Đang xử lý trang 11...
Đang xử lý trang 12...
Đang xử lý trang 13...
Đang xử lý trang 14...
Đang xử lý trang 15...
Đang xử lý trang 16...
Đang xử lý trang 17...
Đang xử lý trang 18...
Đang xử lý trang 19...
Đang xử lý trang 20...
Đang xử lý trang 21...
Đang xử lý trang 22...
Đang xử lý trang 23...
Đang xử lý trang 24...
Đang xử lý trang 25...
Đang xử lý trang 26...
Đang xử lý trang 27...
Đang xử lý trang 28...
Đang xử lý trang 29...
Đang xử lý trang 30...
Đang xử lý trang 31...
Đang xử lý trang 32...
Đang xử lý trang 33...
Đang xử lý trang 34...
Đang xử lý trang 35...
Đang xử lý trang 36...
Đang xử lý trang 37...
Đang xử lý trang 38...
Đang xử lý trang 39...
Đang xử lý trang 40...
Đang xử lý trang 41...
Đang xử lý trang 42...
Đang xử lý trang 43...
Đang xử lý trang 44.

In [ ]:
vnm_lsg = crawl_data_from_page(url_VNM_lsg)
vnm_td = crawl_data_from_page(url_VNM_td)   
vnm_kn = crawl_data_from_page(url_VNM_kn)
vnm_tkdl = crawl_data_from_page(url_VNM_tkdl)

MB

In [ ]:
mb_lsg = crawl_data_from_page(url_MB_lsg)
mb_td = crawl_data_from_page(url_MB_td)
mb_kn = crawl_data_from_page(url_MB_kn)
mb_tkdl = crawl_data_from_page(url_MB_tkdl)

Hòa Phát

In [ ]:
hpg_lsg = crawl_data_from_page(url_HPG_lsg)
hpg_td = crawl_data_from_page(url_HPG_td)
hpg_kn = crawl_data_from_page(url_HPG_kn)
hpg_tkdl = crawl_data_from_page(url_HPG_tkdl)

Hoàng anh gia lai

In [ ]:
hag_tkdl = crawl_data_from_page(url_HAG_tkdl)

Đang xử lý trang 1...
Đang xử lý trang 2...
Đang xử lý trang 3...
Đang xử lý trang 4...
Đang xử lý trang 5...
Đang xử lý trang 6...
Đang xử lý trang 7...
Đang xử lý trang 8...


In [ ]:
hag_lsg = crawl_data_from_page(url_HAG_lsg)
hag_td = crawl_data_from_page(url_HAG_td)
hag_kn = crawl_data_from_page(url_HAG_kn)
hag_tkdl = crawl_data_from_page(url_HAG_tkdl)

In [ ]:
# Tạo DataFrame và lưu
df = pd.DataFrame(all_data, columns=[
    "Ngày", "Giá đóng cửa", "Giá điều chỉnh", "Thay đổi", "KL khớp lệnh", "GT khớp lệnh (tỷ)",
    "KL thỏa thuận", "GT thỏa thuận (tỷ)", "Giá mở cửa", "Giá cao nhất", "Giá thấp nhất"
])
df["Ngày"] = pd.to_datetime(df["Ngày"], format="%d/%m/%Y", errors="coerce")
df = df.dropna(subset=["Ngày"])
#df.to_csv("lich_su_gia_VNM.csv", index=False)

Khối ngoại

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Cấu hình selenium
options = Options()
options.add_argument("--headless")  # Có thể bỏ nếu muốn xem trình duyệt
service = Service(r"C:\Users\dongh\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# Truy cập URL
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-3.chn#data"
driver.get(url)
time.sleep(3)

all_data = []

for page in range(1, 228):
    try:
        print(f"Đang xử lý trang {page}...")

        # Click vào trang hiện tại (trừ trang 1 đã load sẵn)
        if page > 1:
            pagination_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, f"//div[@onclick='ownerCDL.handleChangePage({page})']"))
            )
            pagination_button.click()
            time.sleep(2)  # Đảm bảo nội dung được cập nhật

        # Chờ bảng dữ liệu hiển thị
        wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        table = soup.find("table", {"id": "owner-contents-table"})
        rows = table.find_all("tr")

        for row in rows:
            cols = row.find_all("td")
            if not cols:
                continue
            values = [td.text.strip().replace(",", "").replace("--", "") for td in cols]
            all_data.append(values)

    except Exception as e:
        print(f"Lỗi ở trang {page}: {e}")
        continue

driver.quit()

# Tạo DataFrame và lưu
df = pd.DataFrame(all_data, columns=[
    "Ngày", "Thay đổi", "KL giao dịch ròng", "GT giao dịch ròng", "KL mua", "GT mua",
    "KL bán", "GT bán", "Room còn lại", "Giá cao nhất", "Giá thấp nhất"
])
df["Ngày"] = pd.to_datetime(df["Ngày"], format="%d/%m/%Y", errors="coerce")
df = df.dropna(subset=["Ngày"])
#df.to_csv("lich_su_gia_VNM.csv", index=False)


In [ ]:
df.to_csv(r"khoi_ngoai.csv", index=True)

Tự doanh

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Cấu hình selenium
options = Options()
options.add_argument("--headless")  # Có thể bỏ nếu muốn xem trình duyệt
service = Service(r"C:\Users\dongh\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# Truy cập URL
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-fpt-4.chn#data"
driver.get(url)
time.sleep(3)

all_data = []

for page in range(1, 32):
    try:
        print(f"Đang xử lý trang {page}...")

        # Click vào trang hiện tại (trừ trang 1 đã load sẵn)
        try:
            if page > 1:
                pagination_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[@onclick='tuDoanh.handleChangePage({page})']"))
                )
                pagination_button.click()
                time.sleep(2)  # Đảm bảo nội dung được cập nhật
        except Exception as e:
            print(f"Lỗi khi click vào trang {page}: {e}")
            continue
        # Chờ bảng dữ liệu hiển thị
        wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        table = soup.find("table", {"id": "owner-contents-table"})
        rows = table.find_all("tr")

        for row in rows:
            cols = row.find_all("td")
            if not cols:
                continue
            values = [td.text.strip().replace(",", "").replace("--", "") for td in cols]
            all_data.append(values)

    except Exception as e:
        print(f"Lỗi ở trang {page}: {e}")
        continue

driver.quit()

# Tạo DataFrame và lưu
df = pd.DataFrame(all_data, columns=[
    "Mã", "Ngày", "KL mua", "GT mua",
    "KL bán", "GT bán", "KL giao dịch ròng", "GT giao dịch ròng"
])
df["Ngày"] = pd.to_datetime(df["Ngày"], format="%d/%m/%Y", errors="coerce")
df = df.dropna(subset=["Ngày"])
#df.to_csv("lich_su_gia_VNM.csv", index=False)


In [ ]:
df.to_csv(r"tu_doanh.csv", index=True)

Gộp 3 bảng

In [ ]:
import pandas as pd

df1 = pd.read_csv(r"lich_su_gia_FPT.csv", index_col=0)
df2 = pd.read_csv(r"khoi_ngoai.csv", index_col=0)
df3 = pd.read_csv(r"tu_doanh.csv", index_col=0)

df1 = df1.rename(columns=lambda x: f"lich_su_gia_({x})" if x != "Ngày" else "Ngày")
df2 = df2.rename(columns=lambda x: f"khoi_ngoai_({x})" if x != "Ngày" else "Ngày")
df3 = df3.rename(columns=lambda x: f"tu_doanh_({x})" if x != "Ngày" else "Ngày")


all_data = pd.merge(df1, df2, on="Ngày", how="outer")
all_data = pd.merge(all_data, df3, on="Ngày", how="outer")